In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torchvision import transforms
import numpy as np
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision.models as tv_models
from torchvision.models import ResNet50_Weights
import builtins

# Set all print statements to flush=True
original_print = builtins.print
def print(*args, **kwargs):
    kwargs.setdefault('flush', True)
    original_print(*args, **kwargs)

mem_scores = [{},{},{}]
for i,dataset in enumerate(['train','val','test']):
    with open(f'./lamem/splits/{dataset}_1.txt') as f:
        for line in f:
            image_id, mem_score = line.strip().split(' ')
            mem_scores[i][image_id] = float(mem_score)

train_mem_scores, val_mem_scores, test_mem_scores = mem_scores

class Net(nn.Module):
    def __init__(self):
        super().__init__()        
        #use resnet50 as the base model
        #self.resnet50 = tv_models.resnet50(pretrained=True) #old Pytorch
        self.resnet50 = tv_models.resnet50()
        #modified the last layer for binary classification  
        self.resnet50.fc=torch.nn.Linear(2048, 1) #new layer
    
    def forward(self,x):
        z = self.resnet50(x)
        z = z.view(-1)
        return z

model = Net()
device = 'cuda:0'# if torch.cuda.is_available() else 'cpu'
model.to(device)
print('model created')

# mse = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)
batch_size = 32
version = 0


# Calculate mean and standard deviation of training set target values
train_mem_scores_values = list(train_mem_scores.values())
mean_score = torch.tensor(train_mem_scores_values).mean()
std_score = torch.tensor(train_mem_scores_values).std()
print('mean score',mean_score,'std score',std_score)

# Normalize target values
def normalize_scores(scores):
    return (scores - mean_score) / std_score

# Preprocessing pipeline
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define a different loss function (e.g., Huber loss)
huber_loss = nn.SmoothL1Loss()

def load_and_preprocess_image(image_path):
    image = Image.open(image_path)
    return preprocess(image)

print('train val test',len(list(train_mem_scores.keys())), len(list(val_mem_scores.keys())), len(list(test_mem_scores.keys())))

model created
mean score tensor(0.7565) std score tensor(0.1239)
train val test 45000 3741 10000


In [ ]:
best_val_loss = 999999
train_l = []
val_l = []
with open(f'log_resnet50_LS_{version}.txt','w') as f:
    for epoch in range(100):
        # train
        model.train()
        train_image_ids = list(train_mem_scores.keys())
        train_mem_scores_list = list(train_mem_scores.values())
        num_train_batches = len(train_image_ids) // batch_size
        avg_train_loss = 0
        for i in range(num_train_batches):
            if i % 100 == 0:
                print(f'train {100*i/num_train_batches:.1f}%')
                f.write(f'train {100*i/num_train_batches:.1f}%\n')
                f.flush()
            batch_ids = train_image_ids[i*batch_size:(i+1)*batch_size]
            batch_scores = torch.tensor(train_mem_scores_list[i*batch_size:(i+1)*batch_size], dtype=torch.float32).to(device)


            batch_images = torch.stack([load_and_preprocess_image(f"./lamem/images/{image_id}") for image_id in batch_ids]).to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            outputs = model(batch_images)
            loss = huber_loss(outputs, normalize_scores(batch_scores))
            loss.backward()
            optimizer.step()

            avg_train_loss += loss.item()
    #         break
        avg_train_loss /= num_train_batches
        train_l.append(avg_train_loss)
        print(f'Epoch {epoch + 1}, Average Training Loss: {avg_train_loss}')
        f.write(f'Epoch {epoch + 1}, Average Training Loss: {avg_train_loss}\n')
        f.flush()

        # val
        model.eval()
        val_image_ids = list(val_mem_scores.keys())
        val_mem_scores_list = list(val_mem_scores.values())
        num_val_batches = len(val_image_ids) // batch_size
        with torch.no_grad():
            avg_val_loss = 0
            for i in range(num_val_batches):
                if i % 100 == 0:
                    print(f'val {100*i/num_val_batches:.1f}%')
                    f.write(f'val {100*i/num_val_batches:.1f}%\n')
                    f.flush()
                batch_ids = val_image_ids[i*batch_size:(i+1)*batch_size]
                batch_scores = torch.tensor(val_mem_scores_list[i*batch_size:(i+1)*batch_size], dtype=torch.float32).to(device)

                batch_images = torch.stack([load_and_preprocess_image(f"./lamem/images/{image_id}") for image_id in batch_ids]).to(device)

                # Forward pass
                outputs = model(batch_images)
                loss = huber_loss(outputs, normalize_scores(batch_scores))

                avg_val_loss += loss.item()
    #             break

            avg_val_loss = avg_val_loss / num_val_batches
            val_l.append(avg_val_loss)
            print(f'Epoch {epoch + 1}, Average Validation Loss: {avg_val_loss}')
            f.write(f'Epoch {epoch + 1}, Average Validation Loss: {avg_val_loss}\n')
            f.flush()

            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                print('save best epoch',epoch,avg_val_loss)
                f.write(f'save best epoch {epoch} {avg_val_loss}\n')
                f.flush()
                torch.save(model.state_dict(), f'resnet50_LS_{version}.pth')

        scheduler.step(avg_val_loss)


train 0.0%
train 7.1%
train 14.2%
train 21.3%
train 28.4%
train 35.6%
train 42.7%
train 49.8%
train 56.9%
train 64.0%
train 71.1%
train 78.2%
train 85.3%
train 92.5%
train 99.6%
Epoch 1, Average Training Loss: 0.41663518726062637
val 0.0%
val 86.2%
Epoch 1, Average Validation Loss: 0.3985379694093918
save best epoch 0 0.3985379694093918
train 0.0%
train 7.1%
train 14.2%
train 21.3%
train 28.4%
train 35.6%
train 42.7%
train 49.8%
train 56.9%
train 64.0%
train 71.1%
train 78.2%
train 85.3%
train 92.5%
train 99.6%
Epoch 2, Average Training Loss: 0.3928633587458734
val 0.0%
val 86.2%
Epoch 2, Average Validation Loss: 0.3791220483594927
save best epoch 1 0.3791220483594927
train 0.0%
train 7.1%
train 14.2%
train 21.3%
train 28.4%
train 35.6%
train 42.7%
train 49.8%
train 56.9%
train 64.0%
train 71.1%
train 78.2%
train 85.3%
train 92.5%
train 99.6%
Epoch 3, Average Training Loss: 0.3777620646450801
val 0.0%
val 86.2%
Epoch 3, Average Validation Loss: 0.37255169913686553
save best epoch 2 0.3